In [1]:
import sys
import csv
import operator
import pandas as pd
import warnings
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn import svm
from sklearn import preprocessing
from sklearn import utils
from sklearn.neighbors import KNeighborsRegressor
from sklearn import tree
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
np.set_printoptions(threshold=sys.maxsize)
warnings.filterwarnings('ignore')

# Data Load and Clean-Up

In [2]:
#Reading in normal player stats for yrs 2000-2019
normal_stats_file_name = 'data/normal_stats/normal_stats_{}.csv'
normal_stats = []
for i in range(19, 0, -1):
    normal_stats.append(pd.read_csv(normal_stats_file_name.format(i)))

#Reading in advanced player stats for yrs 2014-2019
adv_stats_file_name = 'data/advanced_stats/adv_stats_{}.csv'
adv_stats = []
for i in range(19, 0, -1):
    adv_stats.append(pd.read_csv(adv_stats_file_name.format(i)))

#Reading in team ratings for yrs 2014-2019
team_ratings_file_name = 'data/team_ratings/teams_{}.csv'
team_ratings = []
for i in range(19, 0, -1):
    team_ratings.append(pd.read_csv(team_ratings_file_name.format(i)))
                        
#Reading in mvp voting results for yrs 2014-2018
mvp_votes_file_name = 'data/mvp_votes/mvp_{}.csv'
mvp_votes = ['THIS IS WHAT WE ARE GONNA PREDICT!']
for i in range(18, 0, -1):
    mvp_votes.append(pd.read_csv(mvp_votes_file_name.format(i)))

In [3]:
#helper method to clean player names for data from 'Basketball-Reference'
def clean_player_names(str):
    index_of_slash = str.find('\\')
    str = str[:index_of_slash]
    return str.replace('*', '')

def clean_data(normal_stats, adv_stats, team_ratings, mvp_votes):
    player_vals = []
    team_vals = []
    mvp_vals = []
    for yr in range(19):
        norm = normal_stats[yr].drop_duplicates(subset='Player', keep='first')
        adv = adv_stats[yr].drop_duplicates(subset='Player', keep='first')
        merge = norm.merge(adv, left_on="Player", right_on='Player')
        merge = merge.drop(['Rk_x', 'Rk_y', 'Pos_y', 'Age_y', 'Tm_y', 'G_y', 'MP_y', 'Unnamed: 24', 'Unnamed: 19'], axis=1)
        merge = merge.rename(index=str, columns={"Pos_x": "Pos", "Age_x": "Age", "Tm_x": "Tm", "G_x": "G", "MP_x": "MP"})
        merge['Player'] = merge['Player'].apply(lambda str: clean_player_names(str))
        added_col_players = np.full((merge.count()[0]), (19 - yr))
        merge['Year'] = (added_col_players)
        
        teams_updated = team_ratings[yr]
        added_col_teams = np.full(len(teams_updated), (19 - yr))
        teams_updated['Year'] = (added_col_teams)
        player_vals += [merge]
        team_vals += [teams_updated]
        if (not isinstance(mvp_votes[yr], str)):
            mvp_updated = mvp_votes[yr]
            added_col_mvps = np.full(len(mvp_updated), (19 - yr))
            mvp_updated['Year'] = (added_col_mvps)
            mvp_updated = mvp_updated[['Player', 'First', 'Share', 'Year']]
            mvp_updated['Player'] = mvp_updated['Player'].apply(lambda str: clean_player_names(str))
            mvp_vals += [mvp_updated]
    
        
    player_data = pd.concat(player_vals)
    team_data = pd.concat(team_vals)
    mvp_data = pd.concat(mvp_vals)
    player_data = pd.merge(player_data, mvp_data,  how='left', left_on=['Player','Year'], right_on = ['Player','Year'])
    player_data = player_data.fillna(0)
    player_data = player_data.rename(index=str, columns={"First": "1st Place MVP Votes", "Share": "MVP Score"})
    player_data = player_data[player_data['Player'] != 'Tony Mitchell']
    player_data = player_data[player_data['Player'] != 'Chris Johnson']
    player_data = player_data[player_data['Player'] != 'Marcus Williams']
    player_data = player_data.assign(id=(player_data['Player']).astype('category').cat.codes)
    player_data = player_data.set_index('Year')
    team_data = team_data.set_index('Year')
    return player_data, team_data
    
player_data, team_data = clean_data(normal_stats, adv_stats, team_ratings, mvp_votes)

# Getting Accustomed to the Data

After data clean up, we now have player and team data from the 2000-2001 NBA season to this past season(18-19). The data is formatted as such: starting with year 19 and down, it contains all players who played in each season. Along with each player's counting stats and advanced stats for each season, there are two columns that will be particularly useful for our analysis: 1st Place MVP Votes representing the number of first place votes they received in that season and MVP Score which represents the share of MVP votes they received out of all that were given. Thus, the MVP in a particular year is simply the player with the MAX('MVP Score'). We can check this intuition below:

In [4]:
seventeen_season = player_data[player_data.index == 17]
mvp_17_season = seventeen_season[seventeen_season['MVP Score'] == max(seventeen_season['MVP Score'])]
mvp_17_season

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,1st Place MVP Votes,MVP Score,id
Year,,,,,,,,,,,,,,,,,,,,,
17,Russell Westbrook,PG,28,OKC,81,81,34.6,10.2,24.0,0.425,...,4.6,13.1,0.224,10.9,4.7,15.6,12.4,69.0,0.879,1508


Russell Westbrook was indeed the MVP of the 2016-2017 NBA Season, we now have some useful data where we can extract useful results! Let's play around a bit more to get accustomed to what we have. Below, we can see the results of all seasons (19-14) of my favorite player: Kevin Durant.

In [5]:
my_fave_player = player_data[player_data['Player'] == 'Kevin Durant']
my_fave_player

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,1st Place MVP Votes,MVP Score,id
Year,,,,,,,,,,,,,,,,,,,,,
19,Kevin Durant,PF,30,GSW,78,78,34.6,9.2,17.7,0.521,...,2.9,11.5,0.204,4.2,0.1,4.3,4.3,0.0,0.000,1023
18,Kevin Durant,PF,29,GSW,68,68,34.2,9.3,18.0,0.516,...,2.9,10.4,0.215,5.0,0.7,5.6,4.5,0.0,0.065,1023
17,Kevin Durant,PF,28,GSW,62,62,33.4,8.9,16.5,0.537,...,4.0,12.0,0.278,5.4,2.6,8.0,5.2,0.0,0.002,1023
16,Kevin Durant,SF,27,OKC,72,72,35.8,9.7,19.2,0.505,...,3.5,14.5,0.270,7.0,0.9,7.9,6.4,0.0,0.112,1023
15,Kevin Durant,SF,26,OKC,27,27,33.8,8.8,17.3,0.510,...,1.0,4.8,0.252,6.3,-0.2,6.1,1.9,0.0,0.000,1023
14,Kevin Durant,SF,25,OKC,81,81,38.5,10.5,20.8,0.503,...,4.4,19.2,0.295,8.4,0.4,8.8,8.5,119.0,0.986,1023
13,Kevin Durant,SF,24,OKC,81,81,38.5,9.0,17.7,0.510,...,5.3,18.9,0.291,6.2,1.4,7.7,7.6,0.0,0.632,1023
12,Kevin Durant,SF,23,OKC,66,66,38.6,9.7,19.7,0.496,...,3.7,12.2,0.230,4.7,0.5,5.2,4.6,24.0,0.735,1023
11,Kevin Durant,SF,22,OKC,78,78,38.9,9.1,19.7,0.462,...,3.3,12.0,0.189,3.7,-0.8,2.9,3.7,0.0,0.157,1023


Additionally, we can see all the players who received MVP votes in all our data: AKA which players have MVP Scores that are nonzero. NOTE: MVP Votes being 0 does not imply not getting any votes as MVP Votes corresponds to 1st place votes, which are not given to every player who was considered for voting. As you'll see below, there are 83 players since 2000 who have received some share of MVP votes.

In [6]:
players_who_got_mvp_votes = player_data[player_data['MVP Score'] != 0.0].groupby('Player')
print("Number of Players who received MVP Votes: " + str(len(players_who_got_mvp_votes.groups)))
players_who_got_mvp_votes.groups

Number of Players who received MVP Votes: 83


{'Al Jefferson': Int64Index([14], dtype='int64', name='Year'),
 'Allen Iverson': Int64Index([6, 5, 3, 2, 1], dtype='int64', name='Year'),
 "Amar'e Stoudemire": Int64Index([11, 10, 8, 7, 5], dtype='int64', name='Year'),
 'Andrei Kirilenko': Int64Index([4], dtype='int64', name='Year'),
 'Antawn Jamison': Int64Index([8], dtype='int64', name='Year'),
 'Anthony Davis': Int64Index([18, 17, 15], dtype='int64', name='Year'),
 'Anthony Mason': Int64Index([1], dtype='int64', name='Year'),
 'Baron Davis': Int64Index([7, 4], dtype='int64', name='Year'),
 'Ben Wallace': Int64Index([4, 3, 2], dtype='int64', name='Year'),
 'Blake Griffin': Int64Index([15, 14, 11], dtype='int64', name='Year'),
 'Brandon Roy': Int64Index([9], dtype='int64', name='Year'),
 'Carlos Boozer': Int64Index([8, 7], dtype='int64', name='Year'),
 'Carmelo Anthony': Int64Index([14, 13, 10, 8, 7, 4], dtype='int64', name='Year'),
 'Chauncey Billups': Int64Index([10, 9, 7, 6], dtype='int64', name='Year'),
 'Chris Bosh': Int64Index([

As you can see, only 29 players received first place votes over the past 18 seasons, some could argue that these have been the best 29 players over the last few years, but thats a subject for another day!

In [7]:
players_who_got_mvp_first_place = player_data[player_data['1st Place MVP Votes'] != 0.0].groupby('Player')
print("Number of Players who received 1st Place Votes: " + str(len(players_who_got_mvp_first_place.groups)))
players_who_got_mvp_first_place.groups

Number of Players who received 1st Place Votes: 29


{'Allen Iverson': Int64Index([5, 1], dtype='int64', name='Year'),
 "Amar'e Stoudemire": Int64Index([5], dtype='int64', name='Year'),
 'Ben Wallace': Int64Index([3], dtype='int64', name='Year'),
 'Carmelo Anthony': Int64Index([13], dtype='int64', name='Year'),
 'Chauncey Billups': Int64Index([6], dtype='int64', name='Year'),
 'Chris Paul': Int64Index([12, 9, 8], dtype='int64', name='Year'),
 'Chris Webber': Int64Index([1], dtype='int64', name='Year'),
 'Derrick Rose': Int64Index([11], dtype='int64', name='Year'),
 'Dirk Nowitzki': Int64Index([7, 6], dtype='int64', name='Year'),
 'Dwight Howard': Int64Index([11, 10, 9], dtype='int64', name='Year'),
 'Dwyane Wade': Int64Index([9], dtype='int64', name='Year'),
 'Elton Brand': Int64Index([6], dtype='int64', name='Year'),
 'Gary Payton': Int64Index([2], dtype='int64', name='Year'),
 'James Harden': Int64Index([18, 17, 15], dtype='int64', name='Year'),
 'Jason Kidd': Int64Index([2], dtype='int64', name='Year'),
 "Jermaine O'Neal": Int64Index(

# Feature Selection

Next, since our goal is to predict the 2019 MVP, we will turn this into a supervised learning problem. Supervised learning problems involve a simple formula: Y = f(X) where X is our input, Y is our output, and f() is our mapping function. Our goal is to create/train such that our mapping function f() is really good at predicting output(Y) accurately given a particular (X). Thus our model will be as such: X will be a d-dimensional vector representing player stats from a certain year and Y will be our output (a vector containing mvp scores for all players from that year -> the max being the player who should win the MVP in that season.


To train our model, however, it is imperative we select good features, so before we choose some, we will utilize three different feature selection algorithms to determine the best features to use: Feature Dependency based on mutual information regression, Random Forest Regressor feature importance (both thanks to sklearn's feature selection package), and the Pearson coefficient matrix determining what values most correlate to our target of 'MVP Score'<br/>


In [8]:
y = player_data['MVP Score']
X = player_data.drop(['MVP Score', 'Player', 'Tm', 'Pos'], axis=1)
mi = mutual_info_regression(X, y)
mi /= np.max(mi)
indices_mi = np.argwhere(mi > 0.2)
top_features_mutual_info = [X.columns.values.tolist()[i[0]] for i in indices_mi]
top_features_mutual_info

['Age',
 'G',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '2P',
 '2PA',
 'FT',
 'FTA',
 'DRB',
 'AST',
 'STL',
 'TOV',
 'PF',
 'PTS',
 'PER',
 'FTr',
 'USG%',
 'OWS',
 'DWS',
 'WS',
 'WS/48',
 'OBPM',
 'DBPM',
 'BPM',
 'VORP',
 '1st Place MVP Votes']

In [9]:
regr = RandomForestRegressor(max_depth=5, random_state=0, n_estimators=100)
regr.fit(X, y)
random_forest = regr.feature_importances_
r_f = {k: v for k, v in (zip(X.columns.values.tolist(), random_forest))}
sorted_r_f = sorted(r_f.items(), key=operator.itemgetter(1))
top_features_r_f = [tup[0] for tup in sorted_r_f]
top_features_r_f.reverse()
top_features_r_f

['1st Place MVP Votes',
 'WS',
 'VORP',
 'DWS',
 'OWS',
 'PTS',
 'PF',
 'PER',
 'G',
 'DBPM',
 'FG',
 '3PAr',
 'Age',
 'MP',
 'id',
 'USG%',
 'BLK',
 'TOV%',
 'AST%',
 'TS%',
 'FG%',
 'BLK%',
 'eFG%',
 'WS/48',
 'FGA',
 '3P%',
 '2P',
 'FT%',
 'BPM',
 '2P%',
 'FTA',
 'FTr',
 '2PA',
 'TOV',
 'STL',
 'OBPM',
 'GS',
 'DRB%',
 'STL%',
 'ORB%',
 'TRB',
 'AST',
 '3PA',
 'FT',
 'TRB%',
 '3P',
 'DRB',
 'ORB']

In [10]:
corr = player_data.corr()
top_features_pearson = corr['MVP Score'].sort_values(ascending=False).index.tolist()
top_features_pearson

['MVP Score',
 '1st Place MVP Votes',
 'VORP',
 'OWS',
 'WS',
 'FTA',
 'FT',
 'PTS',
 'FG',
 '2P',
 'DWS',
 'TOV',
 'FGA',
 '2PA',
 'AST',
 'PER',
 'USG%',
 'BPM',
 'OBPM',
 'DRB',
 'STL',
 'AST%',
 'GS',
 'MP',
 'TRB',
 'WS/48',
 '3P',
 'BLK',
 '3PA',
 'G',
 'DBPM',
 'TS%',
 'ORB',
 '2P%',
 'DRB%',
 'FG%',
 'eFG%',
 'FTr',
 'PF',
 'FT%',
 'STL%',
 '3P%',
 'TRB%',
 'BLK%',
 'id',
 'Age',
 '3PAr',
 'ORB%',
 'TOV%']

In [11]:
finalized_features = list(set(top_features_mutual_info) & set(top_features_pearson) & set(top_features_r_f))
finalized_features = ['Player', 'Tm', 'id'] + finalized_features + ['3P%', '3PA', 'MVP Score', 'Team Wins', 'Team Rank', 'W/L%']

As you can see above, we have obtained three different sets of "best features" we can use to train our models. Now, we have to remove redundant information from all three and finalize a set of features. The finalized set I chose was: 
 
 as well as we will utilize three team_data features: 'Team Wins', 'Team Rank', 'W/L%'. Additionally, to account for modern NBA play-style and features, I (out of own personal choice) decided to include 3P% and 3PA

In [12]:
finalized_features

['Player',
 'Tm',
 'id',
 'BPM',
 'FGA',
 'USG%',
 'PF',
 'WS/48',
 'VORP',
 '1st Place MVP Votes',
 'PTS',
 'G',
 'DRB',
 '2PA',
 'FTr',
 'MP',
 '2P',
 'DWS',
 'PER',
 'TOV',
 'OBPM',
 'GS',
 'AST',
 'Age',
 'FG%',
 'WS',
 'OWS',
 'FG',
 'STL',
 'DBPM',
 'FTA',
 'FT',
 '3P%',
 '3PA',
 'MVP Score',
 'Team Wins',
 'Team Rank',
 'W/L%']

In [16]:
def feature_based_season_vector(player, year):
    team_data_conv = ['MIL', 'GSW', 'TOR', 'UTA', 'HOU', 'POR', 'DEN', 'BOS', 'OKC', 'IND', 'PHI', 'SAS', 'LAC', 'ORL','MIA', 
                  'BRK', 'DET', 'SAC', 'DAL', 'MIN', 'NOP', 'LAL', 'CHA', 'MEM', 'WAS', 'ATL', 'CHI', 'PHO', 'NYK', 'CLE']
    team_conv = {k: v for k, v in (zip(team_data_conv, team_data['Team']))}
    team_conv['NJN'] = 'New Jersey Nets'
    team_conv['SEA'] = 'Seattle Supersonics'
    team_conv['CHO'] = 'Charlotte Bobcats'
    team_conv['CHH'] = 'Charlotte Hornets'
    team_conv['NOH'] = 'New Orleans Hornets'
    team_conv['NOK'] = 'New Orleans/Oklahoma City Hornets'
    team_conv['VAN'] = 'Vancouver Grizzlies'
    this_guys_data = player_data[player_data['Player'] == player]
    this_guys_data = this_guys_data[this_guys_data.index == year]
    this_guys_team = this_guys_data['Tm'][year]
    season_vec = this_guys_data[finalized_features[:len(finalized_features) - 3]]
    if this_guys_data['Tm'].any() == 'TOT':
        season_vec['Team Wins'] = 0
        season_vec['Team Rank'] = 30
        season_vec['W/L%'] = 0.0
    else:
        his_team_data = team_data[team_data['Team'] == team_conv[str(this_guys_team)]]
        season_vec['Team Wins'] = his_team_data['W']
        season_vec['Team Rank'] = his_team_data['Rk']
        season_vec['W/L%'] = his_team_data['W/L%']
    return season_vec
    

Now, with the two helper functions above, we can extract season vectors for a player during a particular season that he played in. For example, Stephen Curry's season vector for 2017 is displayed below.

In [17]:
steph_17 = feature_based_season_vector('Stephen Curry', 17)
steph_17

,Player,Tm,id,BPM,FGA,USG%,PF,WS/48,VORP,1st Place MVP Votes,...,STL,DBPM,FTA,FT,3P%,3PA,MVP Score,Team Wins,Team Rank,W/L%
Year,,,,,,,,,,,,,,,,,,,,,
17,Stephen Curry,GSW,1606,7.3,18.3,30.1,2.3,0.229,6.2,0.0,...,1.8,-0.4,4.6,4.1,0.411,10.0,0.051,67,1,0.817


In [18]:
#||||| THIS METHOD TAKES A REALLY LONG TIME SO ITS COMMENTED OUT RN! |||||
def calculate_all_season_vectors(df):
    s_v = []
    for index, row in df.iterrows():
        print(row[0], index)
        s_v += [feature_based_season_vector(row[0], index).values.tolist()[0]]
    return s_v 
season_vectors = calculate_all_season_vectors(player_data)
print(season_vectors)
np.save('season_vectors', season_vectors)

Alex Abrines 19
Quincy Acy 19
Jaylen Adams 19
Steven Adams 19
Bam Adebayo 19
Deng Adel 19
DeVaughn Akoon-Purcell 19
LaMarcus Aldridge 19
Rawle Alkins 19
Grayson Allen 19
Jarrett Allen 19
Kadeem Allen 19
Al-Farouq Aminu 19
Justin Anderson 19
Kyle Anderson 19
Ryan Anderson 19
Ike Anigbogu 19
Giannis Antetokounmpo 19
Kostas Antetokounmpo 19
Carmelo Anthony 19
OG Anunoby 19
Ryan Arcidiacono 19
Trevor Ariza 19
D.J. Augustin 19
Deandre Ayton 19
Dwayne Bacon 19
Marvin Bagley 19
Ron Baker 19
Wade Baldwin 19
Lonzo Ball 19
Mohamed Bamba 19
J.J. Barea 19
Harrison Barnes 19
Will Barton 19
Keita Bates-Diop 19
Nicolas Batum 19
Jerryd Bayless 19
Aron Baynes 19
Kent Bazemore 19
Bradley Beal 19
Malik Beasley 19
Michael Beasley 19
Marco Belinelli 19
Jordan Bell 19
DeAndre' Bembry 19
Dragan Bender 19
Dairis Bertans 19
Davis Bertans 19
Patrick Beverley 19
Khem Birch 19
Bismack Biyombo 19
Nemanja Bjelica 19
Antonio Blakeney 19
Eric Bledsoe 19
Jaron Blossomgame 19
Bogdan Bogdanovic 19
Bojan Bogdanovic 19
An

Evan Turner 19
Myles Turner 19
Ekpe Udoh 19
Tyler Ulis 19
Jonas Valanciunas 19
Jarred Vanderbilt 19
Fred VanVleet 19
Noah Vonleh 19
Nikola Vucevic 19
Dwyane Wade 19
Moritz Wagner 19
Dion Waiters 19
Kemba Walker 19
Lonnie Walker 19
John Wall 19
Tyrone Wallace 19
Taurean Waller-Prince 19
Brad Wanamaker 19
T.J. Warren 19
Julian Washburn 19
Yuta Watanabe 19
Thomas Welsh 19
Russell Westbrook 19
Derrick White 19
Okaro White 19
Hassan Whiteside 19
Andrew Wiggins 19
Alan Williams 19
C.J. Williams 19
Johnathan Williams 19
Kenrich Williams 19
Lou Williams 19
Marvin Williams 19
Robert Williams 19
Troy Williams 19
D.J. Wilson 19
Justise Winslow 19
Christian Wood 19
Delon Wright 19
Guerschon Yabusele 19
Nick Young 19
Thaddeus Young 19
Trae Young 19
Cody Zeller 19
Tyler Zeller 19
Ante Zizic 19
Ivica Zubac 19
Alex Abrines 18
Quincy Acy 18
Steven Adams 18
Bam Adebayo 18
Arron Afflalo 18
Cole Aldrich 18
LaMarcus Aldridge 18
Jarrett Allen 18
Kadeem Allen 18
Tony Allen 18
Al-Farouq Aminu 18
Justin Anders

Domantas Sabonis 18
JaKarr Sampson 18
Dario Saric 18
Tomas Satoransky 18
Dennis Schroder 18
Mike Scott 18
Thabo Sefolosha 18
Wayne Selden 18
Ramon Sessions 18
Iman Shumpert 18
Pascal Siakam 18
Xavier Silas 18
Ben Simmons 18
Jonathon Simmons 18
Kobi Simmons 18
Kyle Singler 18
Marcus Smart 18
Dennis Smith 18
Ish Smith 18
J.R. Smith 18
Jason Smith 18
Josh Smith 18
Tony Snell 18
Marreese Speights 18
Nik Stauskas 18
Lance Stephenson 18
David Stockton 18
Julyan Stone 18
Edmond Sumner 18
Caleb Swanigan 18
Jayson Tatum 18
Isaiah Taylor 18
Jeff Teague 18
Marquis Teague 18
Mirza Teletovic 18
Garrett Temple 18
Milos Teodosic 18
Jason Terry 18
Daniel Theis 18
Isaiah Thomas 18
Lance Thomas 18
Klay Thompson 18
Tristan Thompson 18
Sindarius Thornwell 18
Anthony Tolliver 18
Karl-Anthony Towns 18
P.J. Tucker 18
Evan Turner 18
Myles Turner 18
Ekpe Udoh 18
Tyler Ulis 18
Jonas Valanciunas 18
Denzel Valentine 18
Fred VanVleet 18
Rashad Vaughn 18
Noah Vonleh 18
Nikola Vucevic 18
Dwyane Wade 18
Dion Waiters 

Rajon Rondo 17
Derrick Rose 17
Terrence Ross 17
Terry Rozier 17
Ricky Rubio 17
Damjan Rudez 17
Brandon Rush 17
D'Angelo Russell 17
Domantas Sabonis 17
Larry Sanders 17
Dario Saric 17
Tomas Satoransky 17
Dennis Schroder 17
Luis Scola 17
Mike Scott 17
Thabo Sefolosha 17
Wayne Selden 17
Kevin Seraphin 17
Ramon Sessions 17
Iman Shumpert 17
Pascal Siakam 17
Jonathon Simmons 17
Kyle Singler 17
Marcus Smart 17
Ish Smith 17
J.R. Smith 17
Jason Smith 17
Tony Snell 17
Marreese Speights 17
Tiago Splitter 17
Nik Stauskas 17
Lance Stephenson 17
Jarnell Stokes 17
Diamond Stone 17
Rodney Stuckey 17
Jared Sullinger 17
Edy Tavares 17
Isaiah Taylor 17
Jeff Teague 17
Mirza Teletovic 17
Garrett Temple 17
Jason Terry 17
Isaiah Thomas 17
Lance Thomas 17
Hollis Thompson 17
Klay Thompson 17
Tristan Thompson 17
Marcus Thornton 17
Mike Tobey 17
Anthony Tolliver 17
Axel Toupane 17
Karl-Anthony Towns 17
P.J. Tucker 17
Evan Turner 17
Myles Turner 17
Beno Udrih 17
Tyler Ulis 17
Jarrod Uthoff 17
Jonas Valanciunas 17

Terrence Ross 16
Terry Rozier 16
Ricky Rubio 16
Damjan Rudez 16
Brandon Rush 16
D'Angelo Russell 16
Robert Sacre 16
JaKarr Sampson 16
Dennis Schroder 16
Luis Scola 16
Mike Scott 16
Thabo Sefolosha 16
Kevin Seraphin 16
Ramon Sessions 16
Iman Shumpert 16
Jonathon Simmons 16
Henry Sims 16
Kyle Singler 16
Donald Sloan 16
Marcus Smart 16
Greg Smith 16
Ish Smith 16
J.R. Smith 16
Jason Smith 16
Josh Smith 16
Russ Smith 16
Tony Snell 16
Marreese Speights 16
Tiago Splitter 16
Nik Stauskas 16
Lance Stephenson 16
Alex Stepheson 16
Jarnell Stokes 16
Amar'e Stoudemire 16
Rodney Stuckey 16
Jared Sullinger 16
Edy Tavares 16
Jeff Teague 16
Mirza Teletovic 16
Garrett Temple 16
Jason Terry 16
Isaiah Thomas 16
Lance Thomas 16
Hollis Thompson 16
Jason Thompson 16
Klay Thompson 16
Tristan Thompson 16
Marcus Thornton 16
Anthony Tolliver 16
Axel Toupane 16
Karl-Anthony Towns 16
P.J. Tucker 16
Evan Turner 16
Myles Turner 16
Beno Udrih 16
Jonas Valanciunas 16
Anderson Varejao 16
Greivis Vasquez 16
Rashad Vaugh

...

Jason Maxiell 6
Sean May 6
Rashad McCants 6
Walter McCarty 6
Antonio McDyess 6
Tracy McGrady 6
Jeff McInnis 6
Aaron McKie 6
Keith McLeod 6
Stanislav Medvedenko 6
Chris Mihm 6
Aaron Miles 6
C.J. Miles 6
Darius Miles 6
Darko Milicic 6
Andre Miller 6
Brad Miller 6
Mike Miller 6
Yao Ming 6
Cuttino Mobley 6
Nazr Mohammed 6
Sergei Monia 6
Mikki Moore 6
Terence Morris 6
Alonzo Mourning 6
Troy Murphy 6
Lamond Murray 6
Ronald Murray 6
Dikembe Mutombo 6
Boniface N'Dong 6
Bostjan Nachbar 6
Lee Nailon 6
Eduardo Najera 6
Steve Nash 6
Jameer Nelson 6
Rasho Nesterovic 6
Ira Newble 6
Andres Nocioni 6
Moochie Norris 6
Dirk Nowitzki 6
Jermaine O'Neal 6
Shaquille O'Neal 6
Fabricio Oberto 6
Lamar Odom 6
Emeka Okafor 6
Mehmet Okur 6
Kevin Ollie 6
Michael Olowokandi 6
Greg Ostertag 6
Bo Outlaw 6
Travis Outlaw 6
Andre Owens 6
Zaza Pachulia 6
Scott Padgett 6
Milt Palacio 6
Jannero Pargo 6
Smush Parker 6
Tony Parker 6
Ruben Patterson 6
Chris Paul 6
Sasha Pavlovic 6
Gary Payton 6
Kendrick Perkins 6
Morris Peter

Tony Parker 5
Ruben Patterson 5
Sasha Pavlovic 5
Gary Payton 5
Anthony Peeler 5
Kirk Penney 5
Kendrick Perkins 5
Wesley Person 5
Morris Peterson 5
Eric Piatkowski 5
Paul Pierce 5
Mickael Pietrus 5
Zoran Planinic 5
Pavel Podkolzin 5
Scot Pollard 5
Mark Pope 5
James Posey 5
Vitaly Potapenko 5
Tayshaun Prince 5
Laron Profit 5
Joel Przybilla 5
Vladimir Radmanovic 5
Aleksandar Radojevic 5
Peter John Ramos 5
Zach Randolph 5
Theo Ratliff 5
Zeljko Rebraca 5
Michael Redd 5
Justin Reed 5
Jared Reiner 5
Jason Richardson 5
Quentin Richardson 5
Luke Ridnour 5
Bernard Robinson 5
Clifford Robinson 5
Glenn Robinson 5
Rodney Rogers 5
Jalen Rose 5
Malik Rose 5
Quinton Ross 5
Michael Ruffin 5
Kareem Rush 5
Bryon Russell 5
John Salmons 5
Jamal Sampson 5
Daniel Santiago 5
Brian Scalabrine 5
Ansu Sesay 5
Ha Seung-Jin 5
Paul Shirley 5
Bobby Simmons 5
Brian Skinner 5
Tamar Slay 5
Donta Smith 5
J.R. Smith 5
Jabari Smith 5
Joe Smith 5
Josh Smith 5
Steve Smith 5
Theron Smith 5
Eric Snow 5
Kirk Snyder 5
Darius So

Erick Strickland 4
Rod Strickland 4
Bruno Sundov 4
Bob Sura 4
Mike Sweetney 4
Stromile Swift 4
Wally Szczerbiak 4
Maurice Taylor 4
Jason Terry 4
Etan Thomas 4
Kenny Thomas 4
Kurt Thomas 4
Tim Thomas 4
Jamaal Tinsley 4
Robert Traylor 4
Gary Trent 4
Jeff Trepagnier 4
Cezary Trybanski 4
Jake Tsakalidis 4
Nikoloz Tskitishvili 4
Hedo Turkoglu 4
Ime Udoka 4
Nick Van Exel 4
Keith Van Horn 4
Jacque Vaughn 4
Jake Voskuhl 4
Slavko Vranes 4
Dwyane Wade 4
Dajuan Wagner 4
Antoine Walker 4
Samaki Walker 4
Ben Wallace 4
Gerald Wallace 4
John Wallace 4
Rasheed Wallace 4
Luke Walton 4
Charlie Ward 4
Earl Watson 4
Clarence Weatherspoon 4
Chris Webber 4
Bonzi Wells 4
Jiri Welsch 4
David Wesley 4
David West 4
Jahidi White 4
Rodney White 4
Chris Whitney 4
Chris Wilcox 4
Mike Wilks 4
Aaron Williams 4
Alvin Williams 4
Eric Williams 4
Frank Williams 4
Jason Williams 4
Jerome Williams 4
Mo Williams 4
Scott Williams 4
Shammond Williams 4
Corliss Williamson 4
Kevin Willis 4
Loren Woods 4
Qyntel Woods 4
Metta Wor

Dalibor Bagaric 2
Vin Baker 2
Erick Barkley 2
Dana Barros 2
Brent Barry 2
Jon Barry 2
Mengke Bateer 2
Tony Battie 2
Shane Battier 2
Charlie Bell 2
Raja Bell 2
Jonathan Bender 2
Travis Best 2
Mike Bibby 2
Chauncey Billups 2
Mookie Blaylock 2
Corie Blount 2
Mark Blount 2
Calvin Booth 2
Ruben Boumtje-Boumtje 2
Cal Bowdler 2
Bruce Bowen 2
Ryan Bowen 2
Ira Bowman 2
Earl Boykins 2
Michael Bradley 2
Shawn Bradley 2
Elton Brand 2
Terrell Brandon 2
Jamison Brewer 2
Primoz Brezec 2
Chucky Brown 2
Damone Brown 2
Dee Brown 2
Ernest Brown 2
Kedrick Brown 2
Kwame Brown 2
P.J. Brown 2
Randy Brown 2
Tierre Brown 2
Rick Brunson 2
Kobe Bryant 2
Mark Bryant 2
Greg Buckner 2
Jud Buechler 2
Rodney Buford 2
Matt Bullard 2
Mitchell Butler 2
Jason Caffey 2
Marcus Camby 2
Elden Campbell 2
Brian Cardinal 2
Anthony Carter 2
Vince Carter 2
Sam Cassell 2
Kelvin Cato 2
Tyson Chandler 2
Calbert Cheaney 2
Chris Childs 2
Doug Christie 2
Keon Clark 2
Speedy Claxton 2
Mateen Cleaves 2
Derrick Coleman 2
Bimbo Coles 2
Jas

Derrick Dial 1
Michael Dickerson 1
Vlade Divac 1
Michael Doleac 1
Keyon Dooling 1
Sherman Douglas 1
Bryce Drew 1
Chris Dudley 1
Tim Duncan 1
Tyus Edney 1
Kevin Edwards 1
Howard Eisley 1
Obinna Ekezie 1
Khalid El-Amin 1
Mario Elie 1
Sean Elliott 1
LaPhonso Ellis 1
Pervis Ellison 1
Evan Eschmeyer 1
Patrick Ewing 1
Jamie Feick 1
Danny Ferry 1
Michael Finley 1
Derek Fisher 1
Marcus Fizer 1
Danny Fortson 1
Greg Foster 1
Jeff Foster 1
Rick Fox 1
Adonal Foyle 1
Steve Francis 1
Todd Fuller 1
Lawrence Funderburke 1
Ruben Garces 1
Chris Garner 1
Kevin Garnett 1
Dean Garrett 1
Pat Garrity 1
Chris Gatling 1
Matt Geiger 1
Devean George 1
Eddie Gill 1
Kendall Gill 1
Dion Glover 1
Anthony Goldwire 1
Steve Goodrich 1
Brian Grant 1
Gary Grant 1
Horace Grant 1
A.C. Green 1
Adrian Griffin 1
Tom Gugliotta 1
A.J. Guyton 1
Darvin Ham 1
Richard Hamilton 1
Zendon Hamilton 1
Tom Hammonds 1
Anfernee Hardaway 1
Tim Hardaway 1
Ron Harper 1
Matt Harpring 1
Al Harrington 1
Othella Harrington 1
Lucious Harris 1
Jaso

In [19]:
season_vectors = np.load('season_vectors.npy')
season_vectors = pd.DataFrame(season_vectors, index=player_data.index, columns=features)

NameError: name 'features' is not defined

# Train/Test Split

In [ ]:
def create_train_set(list_of_years):
    all_but_max = season_vectors[season_vectors.index.isin(list_of_years)]
    y_train = all_but_max['MVP Score'].values.tolist()
    all_but_max = all_but_max.drop(['Player', 'Tm', 'MVP Score', '1st Place MVP Votes'], axis=1)
    x_train = all_but_max.values.tolist()
    return np.array(x_train), np.array(y_train)

In [ ]:
def create_test_set(prediction_year):
    season = season_vectors[season_vectors.index == prediction_year]
    season = season.drop(['Player', 'Tm', 'MVP Score', '1st Place MVP Votes'], axis=1)
    x_test = season.values.tolist()
    return np.array(x_test)

In [ ]:
prediction_year = 19
list_of_years = list(np.arange(1, prediction_year))
x_train, y_train = create_train_set(list_of_years)
x_test = create_test_set(prediction_year)
x_train = x_train.astype(float)
y_train = y_train.astype(float)
x_test = x_test.astype(float)

In [ ]:
np.save('x_train', x_train)
np.save('y_train', y_train)

In [ ]:
print(x_train.shape, y_train.shape)

# Predicting the MVP for the 2018-2019 Season w/ Different Models

In [ ]:
lin_reg = linear_model.LinearRegression()
lasso = linear_model.LassoCV(alphas=np.logspace(-6, 6, 13))
ridge = linear_model.RidgeCV(alphas=np.logspace(-6, 6, 13))
linear_svr = svm.LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True, intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000, random_state=0, tol=1e-05, verbose=0)
knn_reg = KNeighborsRegressor(n_neighbors = 4)
grad_boost = GradientBoostingRegressor(n_estimators=100, learning_rate=1, max_depth=5, random_state=0)

In [ ]:
where_are_NaNs_train = np.isnan(x_train)
where_are_NaNs_test = np.isnan(x_test)
x_train[where_are_NaNs_train] = 0
x_test[where_are_NaNs_test] = 0

#Linear Regression (normal least squares)
lin_reg.fit(x_train, y_train)
lin_reg_predictions_19 = lin_reg.predict(x_test)

#Lasso Regression w/ Built in Cross Validation
lasso.fit(x_train, y_train)
lasso_predictions_19 = lasso.predict(x_test)

#Ridge Regression w/ Built in Cross Validation
ridge.fit(x_train, y_train)
ridge_predictions_19 = ridge.predict(x_test)

#Linear Support Vector Regression
linear_svr.fit(x_train, y_train)
linear_svr_predictions_19 = linear_svr.predict(x_test)

#K Nearest Neighbors Regressor
knn_reg.fit(x_train, y_train)
knn_reg_predictions_19 = knn_reg.predict(x_test)

# Gradient Boosting Regressor
grad_boost.fit(x_train, y_train)
grad_boost_predictions_19 = grad_boost.predict(x_test)

In [ ]:
def calculate_top_10_predictions(predictions, year):
    results = []
    top_10_ascending = list(np.argsort(predictions)[-10:])
    top_10_ascending.reverse()
    for id in top_10_ascending:
        p_id = x_test[id][0]
        year_results = player_data[player_data.index == year]
        print("Player: " + str(year_results[year_results['id'] == p_id]['Player'][year]) + ", MVP Score: " + str(predictions[id]))
        results += [str(year_results[year_results['id'] == p_id]['Player'][year])]
    return results

In [ ]:
def calculate_top_10_year(year):
    year_res = player_data[player_data.index == year].sort_values('MVP Score', ascending=False)['Player'][:10]
    return year_res.values.tolist()

In [ ]:
print("Linear Regression Predictions for 2018-2019 MVP\n")
top_10_lin_reg_19 = calculate_top_10_predictions(lin_reg_predictions_19, 19)

In [ ]:
print("Lasso Regression Predictions for 2018-2019 MVP\n")
top_10_lasso_19 = calculate_top_10_predictions(lasso_predictions_19, 19)

In [ ]:
print("Ridge Regression Predictions for 2018-2019 MVP\n")
top_10_ridge_19 = calculate_top_10_predictions(ridge_predictions_19, 19)

In [ ]:
print("Linear Support Vector Regression Predictions for 2018-2019 MVP\n")
top_10_linear_svr_19 = calculate_top_10_predictions(linear_svr_predictions_19, 19)

In [ ]:
print("KNN Regressor Predictions for 2018-2019 MVP\n")
top_10_knn_reg_19 = calculate_top_10_predictions(knn_reg_predictions_19, 19)

In [ ]:
print("Gradient Boosting Regressor Predictions for 2018-2019 MVP\n")
top_10_grad_boost_19 = calculate_top_10_predictions(grad_boost_predictions_19, 19)

In [ ]:
def determine_accuracy(year, predictions, real):
    #print("Actual Top 10: " + str(real) + "\n")
    #print("Predicted Top 10: " + str(predictions))
    accuracy = len(set(real) & set(predictions))/len(real)
    #print("Accuracy: " + str(accuracy) + "\n")
    return accuracy 

In [ ]:
nba_mvp_ladder = ['Giannis Antetokounmpo', 'James Harden', 'Paul George', 'Stephen Curry', 'Nikola Jokic',
                 'Joel Embiid', 'Kevin Durant', 'Damian Lillard', 'Kawhi Leonard', 'Russell Westbrook']

In [ ]:
print("Linear Regression Accuracy for 19: " + str(determine_accuracy(19, top_10_lin_reg_19, nba_mvp_ladder)))
print("Lasso Regression Accuracy for 19: " + str(determine_accuracy(19, top_10_lasso_19, nba_mvp_ladder)))
print("Ridge Regression Accuracy for 19: " + str(determine_accuracy(19, top_10_ridge_19, nba_mvp_ladder)))
print("Linear SVR Accuracy for 19: " + str(determine_accuracy(19, top_10_linear_svr_19, nba_mvp_ladder)))
print("KNN Regressor Accuracy for 19: " + str(determine_accuracy(19, top_10_knn_reg_19, nba_mvp_ladder)))
print("Gradient Boosting Accuracy for 19: " + str(determine_accuracy(19, top_10_grad_boost_19, nba_mvp_ladder)))

After running tests with 6 different models (namely, Linear Regression, Lasso Regression, Ridge Regression, Linear SVR, KNN Regression, and Gradient Boosting Regression), we have 6 different sets of MVP predictions for 2019. A common theme we see is that James Harden appears at the top of the list 6/6 times. We can also compare the different models' accuracy versus the NBA.com MVP ladder, since results for voting for the 2018-2019 MVP will be out June 24th. Based on this modeling process, I predict James Harden will be the 2018-2019 NBA Most Valuable Player as he `appeared 6/6 times at the top spot amongst our different models' predictions!

# Checking all Models' Accuracy Against the Last 5 MVP Results

In [ ]:
models = [lin_reg, ridge, lasso, linear_svr, knn_reg, grad_boost]
model_names = ['lin_reg', 'ridge', 'lasso', 'linear_svr', 'knn_reg', 'grad_boost']
ac = [[], [], [], [], [], []]
accuracies = {k: v for k, v in (zip(models, ac))}
max_year = 18
for i in range(5):
    for model in models:
        results = []
        list_of_years = list(np.arange(1, max_year))
        x_train_i, y_train_i = create_train_set(list_of_years)
        x_test_i = create_test_set(max_year)
        x_train_i = x_train_i.astype(float)
        y_train_i = y_train_i.astype(float)
        x_test_i = x_test_i.astype(float)
        where_are_NaNs_train = np.isnan(x_train_i)
        where_are_NaNs_test = np.isnan(x_test_i)
        x_train_i[where_are_NaNs_train] = 0
        x_test_i[where_are_NaNs_test] = 0
        model.fit(x_train_i, y_train_i)
        predictions = model.predict(x_test_i)
        top_10 = list(np.argsort(predictions)[-10:])
        top_10.reverse()
        #print("MVP PREDICTIONS FOR YEAR: " + str(max_year) + "\n")  
        for id in top_10:
            p_id = x_test_i[id][0]
            yr = player_data[player_data.index == max_year]
            #print("Player: " + str(yr[yr['id'] == p_id]['Player'][max_year]) + ", MVP Score: " + str(predictions[id]))
            results += [yr[yr['id'] == p_id]['Player'][max_year]]
        real = calculate_top_10_year(max_year)
        accuracies[model].append([determine_accuracy(max_year, results, real)])
    #print("Accuracy for " + str(model_names[i]) + ": " + str(accuracies[i]/len(accuracies)))
    max_year -= 1

In [ ]:
count = 0
for k, v in accuracies.items():
    accuracy = sum([item for sublist in v for item in sublist])/len(v)
    print("Accuracy for " + str(model_names[count]) + " model: " + str(accuracy))
    count+=1